In [1]:
import os
os.chdir('..')

In [2]:
from matplotlib import pyplot as plt
import numpy as np
from timeit import default_timer
import pandas as pd
import seaborn as sns
from src import detector_descriptor as dd
from src import data as dt
from src import util
from src import keypoint_processing as kpp
from src import experiments as ex
import yaml
import cv2
import seaborn as sns
from IPython.display import display, display_html, HTML

In [3]:
image_set_variance = {
    'bark': 'zoom_rotation',
    'boat': 'zoom_rotation',
    'leuven': 'light',
    'bikes': 'blur',
    'trees': 'blur',
    'wall': 'viewpoint',
    'graf': 'viewpoint',
    'ubc': 'jpeg-compression'
}


with open(os.path.join('./', 'config.yml'), 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    cfg = cfg['default']
print(os.getcwd())
dataset_path = cfg['path']['dataset']
dataset = cfg['current_dataset']
# labels = dt.load_labels(os.path.join(dataset_path, dataset), '.txt')
pckl_path = cfg['path']['pckl']
pckl_name = ''.join([dataset, '_label.pckl'])
label_path = os.path.join(pckl_path, pckl_name)
# dt.dump_data(labels, dump_path)
labels = dt.load_data(label_path)
# dataset_pckl_name = cfg['dataset']['dataset_type']['oxford']['pckl_name']
data_path = os.path.join(pckl_path, ''.join([dataset, '.pckl']))

D:\Programming Projects\python projects\state-of-the-binary-descriptor


In [4]:
descriptor_code = {
    'LATCH': 'LT',
    'LUCID': 'LD',
    'AKAZE': 'AZ',
    'BRIEF': 'BF',
    'BRISK': 'BK',
    'ORB': 'OB',
    'FREAK': 'FK',
    'BOOSTDESC': 'BD'
}

detector_code = {
    'HarrisLaplace': 'HL',
    'AKAZE': 'AZ',
    'KAZE': 'KZ',
    'BRISK': 'BK',
    'FAST': 'FT',
    'AGAST': 'AG',
    'ORB': 'OB',
    'StarDetector': 'SD'
}

In [5]:
def display_side_by_side(dfs_with_captions, cmap='Blues'):
    output = ""
#     combined = dict(zip(captions, dfs))
    for caption, df in dfs_with_captions.items():
        output += df.style\
                    .background_gradient(cmap=cmap)\
                    .set_table_styles(
                       [{
                           'selector': 'th',
                           'props': [
                               ('background-color', 'white'),
                               ('color', 'black'),
                               ('border', '1px solid black')
                           ]
                       },{
                           'selector': 'td',
                           'props': [
                               ('border', '1px solid black')
                           ]
                       },{
                           'selector': 'caption',
                           'props': [
                               ('font-size', '18px'),
                               ('color', 'Black'),
                               ('font-family',r'"Times New Roman", Times, serif'),
                               ('text-align', 'center')
                           ]
                       }])\
                    .format('{:.2}')\
                    .set_caption(caption)\
                    ._repr_html_()
#                     .set_table_attributes("style='display:inline'")\
#         output += "\xa0\xa0\xa0\xa0"
    display(HTML(output))

In [6]:
def get_comb_results(image_set_name, data_path, excluded_des=['KAZE', 'DAISY']):
    image_set_name = image_set_name
    image_set = util.get_image_set(data_path, image_set_name)
    image_nums = (1, 2)
    akaze_included_det = ['KAZE', 'AKAZE']
    comb_accuracy_avg = dict()
    comb_precision_avg = dict()
    comb_recall_avg = dict()
    comb_inlier_ratio_avg = dict()

    for descriptor_name in dd.all_descriptors:
        comb_accuracy_avg[descriptor_code[descriptor_name]] = dict()
        comb_precision_avg[descriptor_code[descriptor_name]] = dict()
        comb_recall_avg[descriptor_code[descriptor_name]] = dict()
        comb_inlier_ratio_avg[descriptor_code[descriptor_name]] = dict()
        for detector_name in dd.all_detectors:
            if descriptor_name in excluded_des:
                continue
            if descriptor_name is 'AKAZE' and detector_name not in akaze_included_det:
                comb_accuracy_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = None
                comb_precision_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = None
                comb_recall_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = None
                comb_inlier_ratio_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = None          
                continue
            avg_accuracy = 0
            avg_precision = 0
            avg_recall = 0
            avg_inlier_ratio = 0
            for image_num in range(2, 7):
                image1 = image_set[f'{image_set_name}_img{1}']
                image2 = image_set[f'{image_set_name}_img{image_num}']
                label_name = f'{image_set_name}_img{image_num}'
                label_homography = labels[label_name]
                matching_results = kpp.get_matching_results((image1, image2), label_homography, detector_name, descriptor_name)

                avg_accuracy += matching_results['Accuracy']
                avg_precision += matching_results['Precision']
                avg_recall += matching_results['Recall']
                avg_inlier_ratio += matching_results['Inlier ratio']

            comb_accuracy_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = round(avg_accuracy/5, 2)
            comb_precision_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = round(avg_precision/5, 2)
            comb_recall_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = round(avg_recall/5, 2)
            comb_inlier_ratio_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = round(avg_inlier_ratio/5, 2)
    return {
        'Average Accuracy':comb_accuracy_avg,
        'Average Precision': comb_precision_avg,
        'Average Recall': comb_recall_avg,
        'Average Inlier Ratio': comb_inlier_ratio_avg
    }

In [18]:
image_set_name = 'bikes'
results = get_comb_results(image_set_name, data_path)

In [9]:
cmaps = [
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
cmaps2 = [
            'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
            'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
            'hot', 'afmhot', 'gist_heat', 'copper']

In [40]:
dfs_with_captions = {key: pd.DataFrame(val) for key, val in results.items()}
display_side_by_side(dfs_with_captions, cmaps[10])

,LT,LD,FK,BD,AZ,BF,BK,OB
AG,0.09,0.31,0.03,0.04,nan,0.12,0.03,0.08
KZ,0.27,0.3,0.2,0.14,0.47,0.34,0.18,0.28
AZ,0.21,0.27,0.21,0.13,0.47,0.37,0.2,0.19
FT,0.09,0.31,0.03,0.04,nan,0.12,0.03,0.09
BK,0.06,0.27,0.09,0.06,nan,0.15,0.1,0.08
OB,0.07,0.2,0.18,0.08,nan,0.13,0.22,0.1
HL,0.1,0.37,0.07,0.06,nan,0.15,0.07,0.11
SD,0.23,0.41,0.16,0.13,nan,0.28,0.15,0.21
,LT,LD,FK,BD,AZ,BF,BK,OB
AG,0.14,0.01,0.04,0.04,nan,0.16,0.04,0.11


In [41]:
image_set_name = 'leuven'
results_leuven = get_comb_results(image_set_name, data_path)

In [42]:
dfs_with_captions = {key: pd.DataFrame(val) for key, val in results_leuven.items()}
display_side_by_side(dfs_with_captions, cmaps[10])

,LT,LD,FK,BD,AZ,BF,BK,OB
AG,0.28,0.19,0.12,0.14,nan,0.3,0.19,0.26
KZ,0.25,0.19,0.18,0.21,0.37,0.29,0.25,0.28
AZ,0.2,0.22,0.18,0.18,0.33,0.32,0.27,0.21
FT,0.29,0.2,0.13,0.15,nan,0.31,0.2,0.27
BK,0.13,0.2,0.16,0.07,nan,0.3,0.2,0.13
OB,0.14,0.14,0.25,0.1,nan,0.23,0.3,0.15
HL,0.14,0.33,0.05,0.06,nan,0.17,0.05,0.11
SD,0.26,0.33,0.17,0.17,nan,0.28,0.19,0.25
,LT,LD,FK,BD,AZ,BF,BK,OB
AG,0.48,0.0,0.23,0.23,nan,0.48,0.36,0.45


In [7]:
image_set_name = 'graf'
results_graf = get_comb_results(image_set_name, data_path)

In [11]:
dfs_with_captions = {key: pd.DataFrame(val) for key, val in results_graf.items()}
display_side_by_side(dfs_with_captions, cmaps[10])

,LT,LD,FK,BD,AZ,BF,BK,OB
AG,0.01,0.2,0.05,0.02,nan,0.04,0.04,0.05
KZ,0.01,0.25,0.06,0.03,0.01,0.03,0.06,0.04
AZ,0.02,0.26,0.06,0.06,0.07,0.04,0.07,0.06
FT,0.01,0.21,0.05,0.02,nan,0.04,0.04,0.05
BK,0.01,0.2,0.07,0.04,nan,0.03,0.07,0.04
OB,0.01,0.15,0.05,0.02,nan,0.02,0.05,0.05
HL,0.01,0.27,0.02,0.02,nan,0.03,0.02,0.04
SD,0.01,0.36,0.06,0.02,nan,0.04,0.06,0.05
,LT,LD,FK,BD,AZ,BF,BK,OB
AG,0.09,0.2,0.74,0.14,nan,0.38,0.42,0.36


In [12]:
image_set_name = 'boat'
results_boat = get_comb_results(image_set_name, data_path)

In [13]:
dfs_with_captions = {key: pd.DataFrame(val) for key, val in results_boat.items()}
display_side_by_side(dfs_with_captions, cmaps[10])

,LT,LD,FK,BD,AZ,BF,BK,OB
AG,0.01,0.25,0.02,0.02,nan,0.03,0.02,0.03
KZ,0.0,0.27,0.04,0.02,0.01,0.03,0.05,0.03
AZ,0.01,0.29,0.05,0.04,0.11,0.03,0.05,0.03
FT,0.01,0.25,0.02,0.02,nan,0.03,0.02,0.03
BK,0.01,0.24,0.06,0.03,nan,0.02,0.07,0.03
OB,0.01,0.2,0.07,0.02,nan,0.02,0.09,0.04
HL,0.0,0.32,0.02,0.02,nan,0.02,0.01,0.03
SD,0.01,0.36,0.05,0.02,nan,0.04,0.06,0.03
,LT,LD,FK,BD,AZ,BF,BK,OB
AG,0.0,0.6,0.24,0.2,nan,0.01,0.25,0.2


In [14]:
image_set_name = 'ubc'
results_ubc = get_comb_results(image_set_name, data_path)

In [15]:
dfs_with_captions = {key: pd.DataFrame(val) for key, val in results_ubc.items()}
display_side_by_side(dfs_with_captions, cmaps[10])

,LT,LD,FK,BD,AZ,BF,BK,OB
AG,0.23,0.14,0.09,0.09,nan,0.25,0.11,0.19
KZ,0.46,0.23,0.39,0.25,0.79,0.51,0.42,0.47
AZ,0.39,0.26,0.38,0.22,0.62,0.57,0.45,0.39
FT,0.25,0.15,0.11,0.1,nan,0.28,0.13,0.21
BK,0.16,0.16,0.23,0.07,nan,0.32,0.27,0.18
OB,0.19,0.11,0.44,0.09,nan,0.24,0.48,0.23
HL,0.32,0.26,0.22,0.15,nan,0.36,0.25,0.33
SD,0.58,0.33,0.43,0.3,nan,0.59,0.45,0.5
,LT,LD,FK,BD,AZ,BF,BK,OB
AG,0.46,0.04,0.25,0.2,nan,0.49,0.27,0.4
